I-A TAD Conjunto Disjunto

In [1]:
import numpy as np

def init_cd(n: int):
    return np.full(n, -1, dtype=int)

print(init_cd(5))


def union(rep_1: int, rep_2: int, p_cd: np.ndarray):
    if p_cd[rep_1] < p_cd[rep_2]:
        p_cd[rep_2] = rep_1
        return rep_1
    elif p_cd[rep_1] > p_cd[rep_2]:
        p_cd[rep_1] = rep_2
        return rep_2
    else:
        p_cd[rep_2] = rep_1
        p_cd[rep_1] -= 1
        return rep_2

test = np.array = [-4, 0, 5, 1, 0, 4, -3, 6, 7]
#print(union(0, 6, test))

def find(ind: int, p_cd: np.ndarray):
    root = ind

    print(p_cd)
    while p_cd[root] >= 0:
        root  = p_cd[root]
    
    print(f"la raíz de {ind}: {root}")

    while p_cd[ind] >= 0:
        a = p_cd[ind]
        p_cd[ind] = root
        ind = a
    return root
    

	
print(test)
print(find(2, test))



[-1 -1 -1 -1 -1]
[-4, 0, 5, 1, 0, 4, -3, 6, 7]
[-4, 0, 5, 1, 0, 4, -3, 6, 7]
la raíz de 2: 0
0


II-A Algoritmo de Kruskall

In [6]:
from queue import PriorityQueue

def create_pq(n: int, l_g: list):
    pq = PriorityQueue()

    for u, v, w in l_g:
        pq.put((w, (u,v)))
    
    return pq
    

def kruskal(n: int, l_g: list):
    pq = create_pq(n, l_g)
    ds = init_cd(n)
    l_t = []

    while not pq.empty():
        _, (u, v) = pq.get()

        print(f"Examining edge {u}, {v}")

        x = find(u, ds)
        y = find(v, ds)

        print(f"Got roots {x}, {y}")  
        
        if x != y:
            print("Doing the union")
            l_t.append((u,v))
            union(x, y, ds)
    
    # ctr = 0
    # for i in range(0, len(ds)-1):
    #     if ds[i] < 0:
    #         ctr +=1
    #     if ctr >=2:
    #         return None
        

    return (n, l_t)


II-B Coste de Kruskal

In [9]:
import random
from typing import List
def complete_graph(n_nodes: int, max_weight=50)-> tuple[int, list]:
    n = n_nodes
    l_g = []

    while n_nodes >= 1:
        for i in range(n_nodes-1):
            l_g.append((i, n_nodes -1, random.randint(1, max_weight)))

        n_nodes -= 1

    
    return (n, l_g)


n, a = complete_graph(15, 55)

print(a)

kruskal(n, a)


def time_kruskal(n_graphs: int, n_nodes_ini: int, n_nodes_fin: int, step: int)-> List:

    

[(0, 14, 18), (1, 14, 17), (2, 14, 24), (3, 14, 41), (4, 14, 40), (5, 14, 16), (6, 14, 15), (7, 14, 6), (8, 14, 41), (9, 14, 19), (10, 14, 48), (11, 14, 26), (12, 14, 15), (13, 14, 4), (0, 13, 46), (1, 13, 27), (2, 13, 33), (3, 13, 36), (4, 13, 41), (5, 13, 19), (6, 13, 28), (7, 13, 31), (8, 13, 37), (9, 13, 47), (10, 13, 17), (11, 13, 1), (12, 13, 33), (0, 12, 44), (1, 12, 8), (2, 12, 39), (3, 12, 31), (4, 12, 8), (5, 12, 11), (6, 12, 2), (7, 12, 35), (8, 12, 37), (9, 12, 27), (10, 12, 15), (11, 12, 39), (0, 11, 9), (1, 11, 11), (2, 11, 16), (3, 11, 33), (4, 11, 32), (5, 11, 42), (6, 11, 3), (7, 11, 29), (8, 11, 44), (9, 11, 3), (10, 11, 48), (0, 10, 44), (1, 10, 7), (2, 10, 4), (3, 10, 43), (4, 10, 20), (5, 10, 33), (6, 10, 17), (7, 10, 5), (8, 10, 49), (9, 10, 17), (0, 9, 29), (1, 9, 10), (2, 9, 44), (3, 9, 40), (4, 9, 8), (5, 9, 32), (6, 9, 12), (7, 9, 29), (8, 9, 18), (0, 8, 30), (1, 8, 47), (2, 8, 21), (3, 8, 37), (4, 8, 32), (5, 8, 4), (6, 8, 50), (7, 8, 19), (0, 7, 33), (1, 7, 

(15,
 [(0, 5),
  (11, 13),
  (4, 7),
  (6, 12),
  (6, 11),
  (9, 11),
  (2, 3),
  (2, 10),
  (3, 6),
  (5, 8),
  (13, 14),
  (1, 7),
  (7, 10),
  (0, 4)])